In [1]:
from tensorflow.keras.metrics import CategoricalAccuracy
import tensorflow as tf
from data_loader import get_loader
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from models.distances import Euclidean_Distance
from models.stn import BilinearInterpolation,Localization
from sklearn.neighbors import KNeighborsClassifier
from time import time
import os

2023-02-08 20:12:17.719744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 20:12:17.845318: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-08 20:12:18.337942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nariman/anaconda3/envs/tf/lib/
2023-02-08 20:12:18.338042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugi

In [2]:
def standardize(img):
    mean = np.mean(img)
    std = np.std(img)
    return (img - mean) / std

In [4]:
encoder = keras.models.load_model(
    'model_files/best_encoders/w_densenet_gtsrb2tt100k_encoder.h5',
    custom_objects={'BilinearInterpolation':BilinearInterpolation,'Localization':Localization},compile=False)

2023-02-08 20:12:34.366914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 20:12:34.822929: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9865 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
loader = get_loader('gtsrb') 
test_generator = loader.get_test_generator(batch=8,dim=64)

Loading 43 all data from datasets/GTSRB


In [10]:
t = iter(test_generator)
[Xs,Xq],y = next(t)
Zs = encoder(Xs)

In [13]:
knn = KNeighborsClassifier(n_neighbors=1,n_jobs=-1,metric='l2')
y_train = to_categorical(np.arange(43),num_classes=43)
knn.fit(Zs,y_train);

In [6]:
acc = 0
start = time()
y_train = to_categorical(np.arange(43),num_classes=43)
knn.fit(Zs,y_train);
pb = tf.keras.utils.Progbar(len(test_generator),verbose=1)
for i,z in enumerate(test_generator):
    [Xs,Xq],y_test = z
    Zq = encoder(Xq)
    acc += knn.score(Zq,y_test)
    values=[('test acc',(acc / (i+1)))]
    pb.add(1,values=values)
end = time() - start

4901/4901 [==============================] - 537s 110ms/step - test acc: 0.8886


In [7]:
time_per_image = end / (len(test_generator) * 8)
fps = int(1/time_per_image)
mean_accuracy = np.round((acc / len(test_generator))*100,2)
print(f'One image KNN inference time is {time_per_image:.4f}s')
print(f'FPS is {fps}')
print(f'KNN accuracy is {mean_accuracy}')

One image KNN inference time is 0.0137s
FPS is 72
KNN accuracy is 88.88


In [8]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
import numpy as np
from tensorflow.keras.utils import to_categorical

In [9]:
import pandas as pd

In [10]:
test_path = 'datasets/GTSRB/GTSRB_Test'
X = []
for f in os.listdir(test_path):
    file_path = os.path.join(test_path,f)
    X += [standardize(img_to_array(load_img(file_path,target_size=(64,64))))]
X = np.array(X)

In [11]:
df = pd.read_csv('datasets/GTSRB/GT-final_test.csv',sep=';')
y_true = to_categorical(np.array(df['ClassId']),num_classes=43)

In [12]:
tp = 0
total_samples = len(X)
pb = tf.keras.utils.Progbar(total_samples,verbose=1)
for i,test_image in enumerate(X):
    z_test = encoder(tf.expand_dims(test_image,axis=0))
    y_pred = knn.predict(z_test)
    if np.argmax(y_pred) == np.argmax(y_true[i]):
        tp += 1
    acc = tp / total_samples
    pb.add(1)

2023-02-08 18:40:20.618103: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-02-08 18:40:20.618157: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at pooling_ops_common.cc:403 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer "localization_2_mp_1" "                 f"(type MaxPooling2D).

{{function_node __wrapped__MaxPool_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:MaxPool]

Call arguments received by layer "localization_2_mp_1" "                 f"(type MaxPooling2D):
  • inputs=tf.Tensor(shape=(1, 64, 64, 3), dtype=float32)

In [19]:
acc

0.9160728424386382